<a href="https://colab.research.google.com/github/CardiacModelling/summer-school-2025/blob/main/Hands_on_session_2_action_potential_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-on session 2: action potential models

In this session, we will

- Download a model and prepare it for use in Myokit
- Investigate how the action potential duration changes with pacing rates
- Look at the effects of increasing or reducing current magnitudes.

## Run this first:

In [1]:
# Hide the installation logs:
%%capture
# If installation fails, comment out the line above and run again

# Install Myokit and Sundials
!apt-get install libsundials-dev
!pip install myokit

# Disable debug messages annoyingly shown on colab
import logging
logging.disable(logging.INFO)

# Import myokit and matplotlib
import myokit
import matplotlib.pyplot as plt

## Importing a CellML model

AP models are rarely written "from scratch".
Instead, a typical study will start by either reimplementing a model based on published equations (very laborious!), or by reusing existing code.

To facilitate reuse, the cell electrophysiology community has developed a model exchange format, called [CellML](https://www.cellml.org/), and an online repository, called [PMR](https://models.physiomeproject.org/welcome).
Many cardiac AP models can be found in PMRs "Electrophysiology" section, or through the search function.

Here, we'll download and import the 2008 model of the rabbit ventricular AP by [Mahajan et al.](https://doi.org/10.1529/biophysj.106.98160).

### Obtaining the CellML file

To use this model in Google Colab, either:

1. Download it from [this PMR page](https://models.physiomeproject.org/exposure/a5586b72d07ce03fc40fc98ee846d7a5), then upload it to Colab using the "Files" tab on the left.
2. Download it directly with `wget`, by uncommenting and running one of the lines below:

In [2]:
# Uncomment to download from PMR
#!wget https://models.physiomeproject.org/workspace/mahajan_shiferaw_sato_baher_olcese_xie_yang_chen_restrepo_karma_garfinkel_qu_weiss_2008/download/e86beea89021fd242713459a5c4b5ea2ba33a445/mahajan_shiferaw_sato_baher_olcese_xie_yang_chen_restrepo_karma_garfinkel_qu_weiss_2008.cellml

In [ ]:
# Uncomment to download from GitHub
#!wget https://raw.githubusercontent.com/CardiacModelling/summer-school-2025/refs/heads/main/resources/mahajan_shiferaw_sato_baher_olcese_xie_yang_chen_restrepo_karma_garfinkel_qu_weiss_2008.cellml

### Importing the model in Myokit

To use the model in Myokit, we'll load the module [`myokit.formats`](https://myokit.readthedocs.io/en/stable/api_formats/index.html) and create a `CellMLImporter`

In [ ]:
# Create a CellML importer
import myokit.formats
importer = myokit.formats.importer('cellml')

# Import the model
model = importer.model('mahajan_shiferaw_sato_baher_olcese_xie_yang_chen_restrepo_karma_garfinkel_qu_weiss_2008.cellml')

print(f'Imported model {model.name()}')

To see what we've imported, we can print the model's equations:

In [ ]:
print(model.code())

Near the very bottom, we spot a component named `cell` that contains a state variable `cell.V` in units of `mV`.

This has a defining equation of the form
\begin{align}
\frac{dV}{dt} = - \frac{1}{C} \sum{I}
\end{align}
(note that the division by C isn't written explicitly here, but introduced by writing all the currents in units of pA/pF).

Let's try running a simulation and plotting this variable.
Unlike before, we won't be using a `protocol` (but see below):

In [ ]:
sim = myokit.Simulation(model)
log = sim.run(1000)

fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot()
ax.set_xlabel('Time (ms)')
ax.set_ylabel('V (mV)')
ax.plot(log.time(), log['cell.V'])
plt.show()

That looks like an action potential!

But as we didn't provide a pacing protocol, where is the stimulus current coming from?
As in most CellML models, this is hardcoded in the model.
In this case, in the same component "`cell`":

In [ ]:
print(model.get('cell').code())

The variable `i_Stim` is defined as a piecewise function, equal to either `0` or `stim_amplitude`, depending on the simulation time and the `stim_x` variables below.

In particular, the variables `stim_duration` and `stim_period` indicate periodic pacing with a 3ms stimulus and 400ms period - fast for a human, but this is a rabbit model.
However, in our 1000ms simulation, we only saw a single AP...

To see what's happened, we plot each point visited by the simulation, and draw vertical lines at 400ms and 800ms, where we expected the second and third stimuli to appear.

In [ ]:
fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot()
ax.set_xlabel('Time (ms)')
ax.set_ylabel('V (mV)')
ax.axvline(400, color='gray', ls='--')
ax.axvline(800, color='gray', ls='--')
ax.plot(log.time(), log['cell.V'], 'x')
plt.show()

As the simulation progresses, the variable step-size solver takes larger and larger steps.
Around 800ms, the steps are much larger than 3ms and so the simulation has missed the short stimulus entirely!

Near 400ms the situation is less clear, but we can zoom in to see the same thing happened:

In [ ]:
fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot()
ax.set_xlabel('Time (ms)')
ax.set_ylabel('V (mV)')
ax.axvline(400, color='gray', ls='--')
ax.axvline(403, color='gray', ls='--')
ax.plot(log.time(), log['cell.V'], 'x')
ax.set_xlim(395, 408)
plt.show()

To fix this, we can _force_ the solver to take smaller steps.
By limiting step size to 3ms, the solver is guaranteed to land on a point within the stimulus.

In [ ]:
sim.reset()
sim.set_max_step_size(3)
log = sim.run(1000)

fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot()
ax.set_xlabel('Time (ms)')
ax.set_ylabel('V (mV)')
ax.plot(log.time(), log['cell.V'])
plt.show()

And in fact, the sudden rapid change in the stimulus current will be detected by the solver, which will backtrack to try to find the exact point where the stimulus starts (and ends).

In [ ]:
fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot()
ax.set_xlabel('Time (ms)')
ax.set_ylabel('V (mV)')
ax.axvline(400, color='gray', ls='--')
ax.axvline(403, color='gray', ls='--')
ax.plot(log.time(), log['cell.V'], 'x')
ax.set_xlim(395, 408)
plt.show()

### Extracting the protocol from the model

Running with a maximum step size is not great for performance.
A better strategy is to modify the CellML file and replace the hard-coded stimulus with an entry point for a Myokit pacing protocol.

This can often be done automatically using the module `myokit.lib.guess`:

In [ ]:
# Extract the protocol from the model, and update the model code
protocol = myokit.lib.guess.remove_embedded_protocol(model)

print(protocol)

In [ ]:
print(model.get('cell').code())

Above, we can see the protocol has been correctly converted.

In the model, `i_Stim` is now defined as `pace * stim_amplitude`, where `pace` will be set by the simulation engine - this is indicated through the `bind` keyword, which we won't cover further here.
The variable `stim_period` is still present, but is no longer used.

Using the updated model and extracted protocol, we no longer need the maximum step size:

In [ ]:
sim = myokit.Simulation(model, protocol)
log = sim.run(1000)

fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot()
ax.set_xlabel('Time (ms)')
ax.set_ylabel('V (mV)')
ax.plot(log.time(), log['cell.V'])
plt.show()

Now we have our model, and are ready to experiment!

### Further reading: Other model sources

Not all models make it into PMR.
By one estimate (based on the list below), around 40% of cardiac AP models are listed, with older models more likely to be included.

Increasingly, authors do provide their original code, a supplements to publications, on lab websites, or in (GitHub) repositories.
A long list of AP models with links to known implementations is maintained at https://github.com/myokit/model-list.
To use these models in Myokit, it is necessary to manually translate the code.

## Changing the pacing rate

Now that we've separated the model from the pacing protocol, we can also easily change the pacing rate.

To do this, we could manually specify a new protocol:

In [15]:
protocol = myokit.parse_protocol('''
  [[protocol]]
  # Level  Start    Length   Period   Multiplier
  1.0      0.0      3.0      600.0    0
  ''')

But it's easier to use the [blocktrain](https://myokit.readthedocs.io/en/stable/api_simulations/Protocol.html#myokit.pacing.blocktrain) method:

In [16]:
period = 600
protocol = myokit.pacing.blocktrain(period=period, duration=3)

✏️ Below, create a simulation using a new protocol with a period of 220ms, and simulate at least 5 APs.

Looking closely at the shape of the APs, you should notice

1. A change in the shape from the 1st to the 5th AP, particularly in the plateau phase
2. A gradual reduction in the action potential duration (APD).

Biologically, this is highly desirable behaviour: the cell is adapting to the new pacing rate!

✏️ Make the changes more clear by creating a plot overlaying the 1st and 5th APs.

💡 Manipulating times is easier with `numpy`. You can use `log = log.npview()` to convert all logged data to numpy.

💡 It's even less work to use the Myokit method [log.split_periodic()](https://myokit.readthedocs.io/en/stable/api_simulations/DataLog.html#myokit.DataLog.split_periodic)

✏️ Create a new simulation with a pacing period of 190ms and simulate 10 APs. What happens during the first AP?

✏️ The speed at which the model adapts is dictated by the most slowly changing state variables. In AP models, these are usually internal ion concentrations (e.g. Na+, K+, Cl-, and most importantly Ca2+). Two important concentration variables in this model are `Na.Na_i` and `Ca.Ca_i`, representing the Na+ and Ca2+ concentrations in the cytoplasm. Using the `log` from the simulation above, plot their evolution over time:

## Pacing to "steady state"

Simulation experiments often start by ensuring the cell has settled into a stable rhythm, and is not adapting to any changes we made (or badly chosen initial conditions).

In mathematical terms, we need to wait for the system to reach a _periodic orbit_.
But in AP modeller's jargon this is (incorrectly) called _reaching a steady state_.

To test if we're in a "steady state", we're going to plot the cytosolic sodium and calcium concentrations again, but with two changes that improve performance:

1. We'll only log the variables we need by setting the `log` argument in our call to `run`.
2. We'll only log _one point per period_, by setting a `log_interval`.


In [ ]:
period = 350
protocol = myokit.pacing.blocktrain(period=period, duration=3)
sim = myokit.Simulation(model, protocol)
log = sim.run(500 * period, log=['Environment.time', 'Ca.Ca_i', 'Na.Na_i'], log_interval=period)

fig = plt.figure(figsize=(15, 9))
ax = fig.add_subplot(2, 1, 1)
ax.set_ylabel('[Na]i (mM)')
ax.plot(log.time(), log['Na.Na_i'], '.')
ax = fig.add_subplot(2, 1, 2)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('[Ca]i (mM)')
ax.plot(log.time(), log['Ca.Ca_i'], '.')
plt.show()


This shows that, even for a modest change in pacing rate, adaptation (in this model) takes hundreds of beats.

We can also spot something strange going on with `[Ca]i` during the first beats. We will revisit this below!

✏️ So far, we have assumed that the model was in a "steady state" when we first loaded it. This is a dangerous assumption! Repeat the above experiment to check.

💡 Hint 1: After running once, repeat with higher precision by calling `sim.set_tolerance(1e-8, 1e-8)`.

💡 Hint 2: When reading the final values on the plots, make sure to check the top-left part of the plot, where matplotlib sometimes places an "offset". The values on the y-axis should all be read as "value" + "offset".

**Note**: The long simulation time needed to bring a cell model into "steady state" is one of the reasons that whole-heart simulations typically avoid models with variable internal concentrations.

## Calculating action potential durations

Now, let's quantify the changes to the AP we saw, by measuring the action potential duration (APD).

To do this, we'll first need to define what an "AP" is.
This can get quite complicated, but for now, we'll stick with the common definition of "any depolarisation above 10% of the voltage range", i.e. the APD90.

We'll start by running a quick simulation to find the lower and upper bounds of the voltage range.

In [ ]:
period = 400
protocol = myokit.pacing.blocktrain(period=period, duration=3)
sim = myokit.Simulation(model, protocol)
log = sim.run(3 * period)

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot()
ax.plot(log.time(), log['cell.V'])
plt.show()

Now we can get the minimum and maximum voltages using `min` and `max`:

In [ ]:
Vmin = min(log['cell.V'])
Vmax = max(log['cell.V'])
print(Vmin, Vmax)

In [ ]:
V90 = Vmin + 0.1 * (Vmax - Vmin)
print('Threshold:', V90)

Using this threshold, we run another simulation, but this time with APD measurement enabled.
We can do this by passing in a `apd_variable` and `apd_threshold` to the `run` method:

In [ ]:
period = 400
protocol = myokit.pacing.blocktrain(period=period, duration=3)
sim = myokit.Simulation(model, protocol)
log, apds = sim.run(3 * period, apd_variable='cell.V', apd_threshold=V90)

print(apds)

Note how this changes the return type of the [run](https://myokit.readthedocs.io/en/stable/api_simulations/Simulation.html#myokit.Simulation.run) method: It now returns _two_ DataLogs, where the second has the entries `start` and `duration` containing the start times and durations of all detected APs.

✏️ Change the pacing frequency to 350ms, and plot APD during the first 1000 beats.

💡 You can speed up the simulation significantly by disabling logging. To do this, add `log=myokit.LOG_NONE` to the run method's arguments.

💡 For the plot's x-axis, you can use `range(len(apds['duration']))` to create a list of the numbers [0, 1, 2, ...] up to the number of APs.

✏️ Repeat this exercise with a period of 200ms. What do you observe?

💡 If the plot looks strange, try zooming in on the final 10 APs with `ax.set_xlim(990, 1000)`.

At this rate, the system doesn't settle into a periodic orbit of 200ms.
Instead, the APD alternates between two stable values: it might still be periodic, but the period has doubled to 400ms!

In cell electrophysiology, this is known as _alternans_ and is of particular interest because of its [association with reentrant arrhythmias](https://www.ahajournals.org/doi/10.1161/JAHA.119.013750).

## Restitution

> Cardiac restitution is the ability of the heart to recover from one beat to the next. At its simplest, it is the relationship of how long the heart is in contraction compared to how much rest it has between beats to permit recovery.

[Fossa, 2017](https://onlinelibrary.wiley.com/doi/epdf/10.1111/anec.12460).

At the _cell level_, the term _restitution_ is used to refer to the relationship between the action potential duration (APD) and diastolic interval (DI).
A common way to visualise this, is to plot the APD as a function of the DI.

✏️ Plot the restitution curve for the Mahajan et al. model, using the periods `[300, 280, 260, 240, 220, 210, 205, 200, 195, 190]`. To get closer to steady state, run for at least 100 beats at each period.

💡 Make sure you log at least 2 APDs at each period, in case of alternans

💡 To improve performance, disable logging with `log=myokit.LOG_NONE`

💡 You can reuse the same simulation for each period, by changing the protocol with `sim.set_protocol()` and resetting time to 0 with `sim.set_time(0)`

## Upsetting the balance in the plateau

During the plateau phase of the AP, the inward current ICaL is balanced by outward potassium currents.
In humans, the strongest potassium current is IKr, but in this rabbit model IKs is dominant.
The size of these currents is determined by model variables `ICaL.gca = 182 [mmol/cm/C]` and `IKs.gks = 0.1386 [mS/uF]`.

✏️ Show the effect of reducing or increasing ICaL and IKs on the AP, by changing the `gca` and `gks` values using [`sim.set_constant`](https://myokit.readthedocs.io/en/stable/api_simulations/Simulation.html#myokit.Simulation.set_constant).

✏️ Show the effect of changing ICaL or IKs size on the restitution curve.